In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = 12, 8
plt.rcParams.update({'font.size': 10})

In [3]:
# Aquifers
Auser_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Aquifer_Auser.csv'
Doganella_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Aquifer_Doganella.csv'
Luco_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Aquifer_Luco.csv'
Petrignano_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Aquifer_Petrignano.csv'

In [4]:
# Lake
Bilancino_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Lake_Bilancino.csv'

In [5]:
# River
Arno_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/River_Arno.csv'

In [6]:
# Water springs
Amiata_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Water_Spring_Amiata.csv'
Lupa_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Water_Spring_Lupa.csv'
Madonna_path = '/Users/gauravtyagi/Downloads/Data/acea-water-prediction/Water_Spring_Madonna_di_Canneto.csv'

In [7]:
# Column names for target variables
targets = {
    'Auser': [
        'Depth_to_Groundwater_SAL',
        'Depth_to_Groundwater_CoS',
        'Depth_to_Groundwater_LT2'
        ],
    'Doganella': [
        'Depth_to_Groundwater_Pozzo_1',
        'Depth_to_Groundwater_Pozzo_2',
        'Depth_to_Groundwater_Pozzo_3',
        'Depth_to_Groundwater_Pozzo_4',
        'Depth_to_Groundwater_Pozzo_5',
        'Depth_to_Groundwater_Pozzo_6',
        'Depth_to_Groundwater_Pozzo_7',
        'Depth_to_Groundwater_Pozzo_8',
        'Depth_to_Groundwater_Pozzo_9'
        ],
    'Luco': [
        'Depth_to_Groundwater_Podere_Casetta'
        ],
    'Petrignano': [
        'Depth_to_Groundwater_P24',
        'Depth_to_Groundwater_P25'
        ],
    'Bilancino': [
        'Lake_Level', 
        'Flow_Rate'
        ],
    'Arno': [
        'Hydrometry_Nave_di_Rosano'
        ],
    'Amiata': [
        'Flow_Rate_Bugnano',
        'Flow_Rate_Arbure',
        'Flow_Rate_Ermicciolo',
        'Flow_Rate_Galleria_Alta'
        ],
    'Lupa': [
        'Flow_Rate_Lupa'
        ],
    'Madonna': [
        'Flow_Rate_Madonna_di_Canneto'
        ]
    }

In [8]:
# Models to be compared
models = [('RandomForest', RandomForestRegressor()),
          ('ExtraTrees', ExtraTreesRegressor()),
          ('GradientBoosting', make_pipeline(StandardScaler(), GradientBoostingRegressor())),
          ('KNeighbors', make_pipeline(StandardScaler(), KNeighborsRegressor()))]

In [9]:
# Splits and shuffle for cross-validation
kf = KFold(3, shuffle=True, random_state=1)

In [10]:
# For applying various data frequencies
resampling = {'monthly': 'M', 'weekly': 'W', 'daily': 'D'}

In [11]:
def plot_nans(df: pd.DataFrame, obj_id: str):
    """Function calculates percentage of missing values by column
    and creates a bar plot."""
    rows, _ = df.shape
    missing_values = df.isna().sum() / rows * 100
    missing_values = missing_values[missing_values != 0]
    missing_values.sort_values(inplace=True)
    title = obj_id + ' missing values'
    plt.barh(missing_values.index, missing_values.values)
    plt.xlabel('Percentage (%)')
    plt.title(title)
    plt.show()

In [12]:
def plot_distribution(df: pd.DataFrame):
    """Function plots a histogram for parameter distribution."""
    df.hist(bins=20, figsize=(14, 10))
    plt.show()

In [13]:
def plot_correlation(df: pd.DataFrame, obj_id: str, targets: list):
    """Function calculates correlation between parameters
    and creates a heatmap."""
    title = obj_id + ' Heatmap'
    targets_correlation = df.corr()[targets]
    ax = sns.heatmap(targets_correlation, center=0, annot=True, cmap='RdBu_r')
    l, r = ax.get_ylim()
    ax.set_ylim(l + 0.5, r - 0.5)
    plt.yticks(rotation=0)
    plt.title(title)
    plt.show()

In [14]:
def plot_timeseries(df: pd.DataFrame, obj_id: str, targets: list):
    """Function plots target variable against the timescale."""
    for target in targets:
        plt.plot(df.index, df[target], label=target)
    title = obj_id + ' actual data'
    plt.legend()
    plt.title(title)
    plt.show()

In [15]:
def plot_seasonality(df: pd.DataFrame, targets: list):
    """Function creates a seasonal decomposition plot for target variables.
    Temporary interpolation of missing values is performed on a resampled
    monthly data, which does not affect the original dataset."""
    for target in targets:
        monthly_interpolated = df[target].resample('M').mean().interpolate(method='akima').dropna()
        
        decomposition = seasonal_decompose(monthly_interpolated)
        observed = decomposition.observed
        trend = decomposition.trend
        seasonal = decomposition.seasonal
        residual = decomposition.resid
        dates = monthly_interpolated.index

        plt.plot(dates, observed, label='Original data')
        plt.plot(dates, trend, label='Trend')
        plt.plot(dates, seasonal, label='Seasonal')
        plt.plot(dates, residual, label='Residual')
        plt.legend()
        plt.title(f'{target} seasonal decomposition')
        plt.tight_layout()
        plt.show()

In [16]:
def data_cleaning(df: pd.DataFrame):
    """Function replaces 0 values with np.nan in all columns except rainfall."""
    for column in df.columns:
        if column.find('Rainfall') == -1:
            df[column] = df[column].apply(lambda x: np.nan if x == 0 else x)
    return df

In [17]:
def get_data(path: str):
    """Function extracts data from a csv file and converts date column
    to datetime index."""
    df = pd.read_csv(path,
                       parse_dates=['Date'],
                       date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    df.dropna(subset=['Date'], inplace=True)  # Madonna_di_Canneto dataset has empty rows
    df.set_index('Date', inplace=True)
    # Remove erroneous 0 values from all columns except rainfall
    df = data_cleaning(df)
    return df

In [18]:
def resample_data(df: pd.DataFrame, freq: str):
    """Function converts daily data into weekly or monthly averages."""
    return df.resample(freq).mean()

In [19]:
def add_seasonality(df: pd.DataFrame):
    """Function adds columns specifying year, month and day of a year
    and binary column for rainy season (October through April)."""
    df['Year'] = df.index.year
    df['Month'] = df.index.month
    df['Week'] = df.index.weekofyear
    df['Day'] = df.index.dayofyear
    df['Rainy_Season'] = df['Month'].apply(lambda x: 0 if 5 <= x <= 9 else 1)
    return df

In [20]:
def add_weekly_averages(df: pd.DataFrame):
    """Function adds weekly rolling average values for rainfall and temperature,
    which are used as additional features in daily datasets."""
    for column in df.columns:
        if column.find('Rainfall') > -1 or column.find('Temperature') > -1:
            df[f'{column}_weekly'] = df[column].rolling(7).mean()
    return df

In [21]:
def select_features(df: pd.DataFrame):
    """Function creates a list of most correlated features for the target."""
    target_correlation = df.corr()['Target']
    mosts_correlated = target_correlation[(target_correlation >= 0.2) | (target_correlation <= -0.2)].index.tolist()
    return mosts_correlated

In [22]:
def get_X_y(df: pd.DataFrame, target: str, steps_ahead: int):
    """Function splits data into input features and target variable,
    returns a tuple containing list of features, X and y."""
    X = df.copy()
    # Add a column that contains target value for the predicted future period
    # (shift target column backwards for required number of periods)
    X['Target'] = X[target].shift(-steps_ahead)
    # Reduce input to the most correlated features
    features = select_features(X)
    X = X[features]
    # Here the last row of actual inputs is lost
    # because there is no future period target value for it
    X.dropna(inplace=True)
    y = X.pop('Target')
    return features[:-1], X, y


In [23]:
def evaluate_models(input_data: pd.DataFrame, target: pd.Series, target_name: str):
    """Function estimates cross-val score for several models.
    If the highest cv score is above 0.6, returns a tuple
    with fitted model and its name, otherwise returns a tuple
    with None and an empty string."""

    best_cv = -1
    best_model = None
    best_model_name = ''

    for name, model in models:
        cv_r2 = cross_val_score(model, input_data, target, cv=kf, scoring='r2').mean()
        cv_mae = - cross_val_score(model, input_data, target, cv=kf, scoring='neg_mean_absolute_error').mean()
        cv_rmse = - cross_val_score(model, input_data, target, cv=kf, scoring='neg_mean_squared_error').mean()
        print(f'{name} cross-val score for {target_name}:\n\tR2 = {cv_r2}\n\tMAE = {cv_mae}\n\tRMSE = {cv_rmse}')

        if cv_r2 > best_cv:
            best_cv = cv_r2
            best_model = model
            best_model_name = name

    if best_cv >= 0.6:
        best_model.fit(input_data, target)
        return best_model_name, best_model
    else:
        return '', None

In [24]:
def simple_prediction(ts: pd.Series, n_periods_1: int, n_perionds_2: int, steps_ahead: int):
    """Function returns a prediction based on the actual value of the target variable
    for the latest period and two linear trend predictions equally weighted."""
    # Actual last value in the time series
    last_period_value = ts.iloc[len(ts) - 1]
    # Linear trend based on n_periods_1
    X_1 = np.array([i for i in range(1, n_periods_1 + 1)]).reshape(-1, 1)
    linear_prediction_1 = LinearRegression().fit(X_1, ts.tail(n_periods_1)).predict([[n_periods_1 + steps_ahead]])[0]
    # Linear trend based on n_periods_2
    X_2 = np.array([i for i in range(1, n_perionds_2 + 1)]).reshape(-1, 1)
    linear_prediction_2 = LinearRegression().fit(X_2, ts.tail(n_perionds_2)).predict([[n_perionds_2 + steps_ahead]])[0]
    # Average of the three values
    prediction = (last_period_value + linear_prediction_1 + linear_prediction_2) / 3
    return prediction

In [25]:
def modelling(df: pd.DataFrame, targets: list, obj_id: str, freq: str, steps_ahead: int):
    """Function preprocesses data, creates models and estimates their accuracy,
    gets prediction for the future period from the best model if R2 >= 0.6
    or uses simple prediction based on the last actual value and linear trends."""
    print(f'\nCreating {freq} model for {obj_id}\n')
    df = resample_data(df, resampling[freq])  # Change data frequency
    if freq == 'daily':
        df = add_weekly_averages(df)  # Add weekly rolling averages as a feature

    # Select input for each target that contains the most correlated features
    for target in targets:
        features, X, y = get_X_y(df, target, steps_ahead)
        model_name, model = evaluate_models(X, y, target)

        if model_name:  # Best R2 >= 0.6
            # Get the actual last row of input data from the dataset
            # (if there are NaNs, get the last row with all required features)
            input_data = df[features].dropna()
            input_date = input_data.index.max()
            input_data = input_data.iloc[len(input_data) - 1, :].values.reshape(1, -1)
            prediction = model.predict(input_data)[0]
        else:  # Low R2
            model_name = 'Average and linear trend'
            features = [target]
            input_data = df[target].dropna()
            input_date = input_data.index.max()
            # Take into account last value and trends of the last 5 and 10 periods
            prediction = simple_prediction(input_data, 5, 10, steps_ahead)

        print(f'\n{model_name} {freq} prediction for {target}: {prediction}')
        print(f'\nInput features: {", ".join(features)}\nInput date: {input_date}')
        print(f'Prediction for {steps_ahead} step(s) ahead.\n')

In [26]:
data = get_data(Auser_path)

In [27]:
target_cols = targets['Auser']

In [28]:
#data.describe()

In [29]:
#plot_nans(data, 'Auser aquifer')

In [30]:
#plot_distribution(data)

In [31]:
#plot_correlation(data, 'Auser Aquifer', target_cols)

In [32]:
#plot_timeseries(data, 'Auser Aquifer', target_cols)

In [33]:
#plot_seasonality(data, target_cols)

In [34]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [35]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Auser Aquifer', 'monthly', 1)


Creating monthly model for Auser Aquifer

RandomForest cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.3953456564433873
	MAE = 0.19360582098203993
	RMSE = 0.0562017267550596
ExtraTrees cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.2076496894320744
	MAE = 0.22238238920032613
	RMSE = 0.0695426058317169
GradientBoosting cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.338403755237274
	MAE = 0.18979169319255382
	RMSE = 0.05258405801972313
KNeighbors cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.21516570980995905
	MAE = 0.2090907329152071
	RMSE = 0.0737483950414068

Average and linear trend monthly prediction for Depth_to_Groundwater_SAL: -5.413183656737939

Input features: Depth_to_Groundwater_SAL
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.

RandomForest cross-val score for Depth_to_Groundwater_CoS:
	R2 = 0.6241689505249083
	MAE = 0.5224786032025704
	RMSE = 0.3901993230185381
ExtraTrees cross-val score for Depth_to_Groundwater_CoS:
	R2 = 0.63

In [36]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Auser Aquifer', 'daily', 1)


Creating daily model for Auser Aquifer

RandomForest cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.9806246426728965
	MAE = 0.025537581848636904
	RMSE = 0.0018076970294231877
ExtraTrees cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.9833018652719391
	MAE = 0.02296206283944967
	RMSE = 0.0015029859792227054
GradientBoosting cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.9844002990348835
	MAE = 0.02496633039972357
	RMSE = 0.0014316691593016084
KNeighbors cross-val score for Depth_to_Groundwater_SAL:
	R2 = 0.9136639928300703
	MAE = 0.05430260731265762
	RMSE = 0.007864142361639858

GradientBoosting daily prediction for Depth_to_Groundwater_SAL: -5.876635794279604

Input features: Depth_to_Groundwater_LT2, Depth_to_Groundwater_SAL, Depth_to_Groundwater_PAG, Depth_to_Groundwater_CoS, Depth_to_Groundwater_DIEC, Temperature_Orentano, Temperature_Monte_Serra, Temperature_Ponte_a_Moriano, Temperature_Lucca_Orto_Botanico, Volume_POL, Volume_CSAL, Hydrometry_Monte_S_Quirico, H

In [37]:
data = get_data(Doganella_path)

In [38]:
target_cols = targets['Doganella']

In [39]:
#data.describe()

In [40]:
#plot_nans(data, 'Doganella aquifer')

In [41]:
#plot_distribution(data)

In [42]:
#plot_correlation(data, 'Doganella Aquifer', target_cols)

In [43]:
#plot_timeseries(data, 'Doganella Aquifer', target_cols)

In [44]:
#plot_seasonality(data, target_cols)

In [45]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [46]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Doganella Aquifer', 'monthly', 1)


Creating monthly model for Doganella Aquifer

RandomForest cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.5091912485818647
	MAE = 3.1172184789650363
	RMSE = 13.139542669659818
ExtraTrees cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.5130984968345417
	MAE = 2.7304035057349387
	RMSE = 9.968550703033683
GradientBoosting cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.20581330251411578
	MAE = 3.294419340570343
	RMSE = 16.48165102379651
KNeighbors cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.15726504512647219
	MAE = 3.7411349162292478
	RMSE = 25.377063781927408

Average and linear trend monthly prediction for Depth_to_Groundwater_Pozzo_1: -49.327901333992486

Input features: Depth_to_Groundwater_Pozzo_1
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.

RandomForest cross-val score for Depth_to_Groundwater_Pozzo_2:
	R2 = -0.30876960776483936
	MAE = 0.3624096245771035
	RMSE = 0.2783472791775274
ExtraTrees cross-val score for Depth_

In [47]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Doganella Aquifer', 'daily', 1)


Creating daily model for Doganella Aquifer

RandomForest cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.981743797864184
	MAE = 0.4057192624244817
	RMSE = 1.8874219884452417
ExtraTrees cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.9815394250236956
	MAE = 0.36112951011590066
	RMSE = 1.870443856141856
GradientBoosting cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.9814786926345693
	MAE = 0.4470682699604826
	RMSE = 1.8749209296148617
KNeighbors cross-val score for Depth_to_Groundwater_Pozzo_1:
	R2 = 0.9762833435537743
	MAE = 0.5882254800750667
	RMSE = 2.3973075326926416

RandomForest daily prediction for Depth_to_Groundwater_Pozzo_1: -50.74339999999998

Input features: Depth_to_Groundwater_Pozzo_1, Depth_to_Groundwater_Pozzo_2, Depth_to_Groundwater_Pozzo_4, Depth_to_Groundwater_Pozzo_5, Depth_to_Groundwater_Pozzo_6, Depth_to_Groundwater_Pozzo_7, Depth_to_Groundwater_Pozzo_8, Year, Month, Week, Day
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) a

In [48]:
data = get_data(Luco_path)

In [49]:
target_cols = targets['Luco']

In [50]:
#data.describe()

In [51]:
#plot_nans(data, 'Luco aquifer')

In [52]:
#plot_distribution(data)

In [53]:
#plot_correlation(data, 'Luco Aquifer', target_cols)

In [54]:
#plot_timeseries(data, 'Luco Aquifer', target_cols)

In [55]:
#plot_seasonality(data, target_cols)

In [56]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [57]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Luco Aquifer', 'monthly', 1)


Creating monthly model for Luco Aquifer

RandomForest cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = -0.17263180918531273
	MAE = 0.192133287115637
	RMSE = 0.059690104075551954
ExtraTrees cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = -0.01835397522464451
	MAE = 0.18773978966232777
	RMSE = 0.06075808482438947
GradientBoosting cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = 0.08925601984289289
	MAE = 0.18518550615119314
	RMSE = 0.03745708234125971
KNeighbors cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = 0.04124798225047207
	MAE = 0.19998050772091416
	RMSE = 0.08425786223447802

Average and linear trend monthly prediction for Depth_to_Groundwater_Podere_Casetta: -7.640500723133997

Input features: Depth_to_Groundwater_Podere_Casetta
Input date: 2019-01-31 00:00:00
Prediction for 1 step(s) ahead.



In [58]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Luco Aquifer', 'daily', 1)


Creating daily model for Luco Aquifer

RandomForest cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = 0.9893171646318493
	MAE = 0.01956287454563774
	RMSE = 0.00132785170448966
ExtraTrees cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = 0.9897524186683335
	MAE = 0.01905083996463756
	RMSE = 0.0013009340308478266
GradientBoosting cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = 0.9889856565078089
	MAE = 0.02112814689609217
	RMSE = 0.0013364899301154221
KNeighbors cross-val score for Depth_to_Groundwater_Podere_Casetta:
	R2 = 0.9826560782205046
	MAE = 0.02726692209450841
	RMSE = 0.0021053541605265836

ExtraTrees daily prediction for Depth_to_Groundwater_Podere_Casetta: -7.600000000000014

Input features: Depth_to_Groundwater_Podere_Casetta, Depth_to_Groundwater_Pozzo_1, Depth_to_Groundwater_Pozzo_3, Temperature_Siena_Poggio_al_Vento, Volume_Pozzo_1, Volume_Pozzo_3, Volume_Pozzo_4, Temperature_Siena_Poggio_al_Vento_weekly, Year, Rainfall_Siena_Poggi

In [59]:
# Example of daily forecast based on correlated features without previous values of the target
data = get_data(Luco_path)
data['Year'] = data.index.year

target = ['Depth_to_Groundwater_Podere_Casetta']
features = ['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_3', 
            'Volume_Pozzo_1', 'Volume_Pozzo_3', 'Volume_Pozzo_4', 'Year']

# Select input and output columns
data = data[features + target]
# Last row with all features without target
input_date = data.index.max()
input_data = data.loc[input_date, features].values.reshape(1, -1)
# Extract y values
data['Depth_to_Groundwater_Podere_Casetta'] = data['Depth_to_Groundwater_Podere_Casetta'].shift(-1)
data.dropna(inplace=True)
y = data.pop('Depth_to_Groundwater_Podere_Casetta')
# Cross-validate the model
model_name, model = models[0]
cv_r2 = cross_val_score(model, data, y, cv=kf, scoring='r2').mean()
cv_mae = - cross_val_score(model, data, y, cv=kf, scoring='neg_mean_absolute_error').mean()
cv_rmse = - cross_val_score(model, data, y, cv=kf, scoring='neg_mean_squared_error').mean()
print(f'{model_name} cross-val score for {target}:\n\tR2 = {cv_r2}\n\tMAE = {cv_mae}\n\tRMSE = {cv_rmse}')
# Train the model
model.fit(data, y)
prediction = model.predict(input_data)[0]
print(f'\n{model_name} daily prediction for {target}: {prediction}')
print(f'\nInput features: {", ".join(features)}\nInput date: {input_date}\n')

RandomForest cross-val score for ['Depth_to_Groundwater_Podere_Casetta']:
	R2 = 0.9695738015071563
	MAE = 0.03566116955882204
	RMSE = 0.003683892690834179

RandomForest daily prediction for ['Depth_to_Groundwater_Podere_Casetta']: -7.564000000000008

Input features: Depth_to_Groundwater_Pozzo_1, Depth_to_Groundwater_Pozzo_3, Volume_Pozzo_1, Volume_Pozzo_3, Volume_Pozzo_4, Year
Input date: 2020-06-30 00:00:00



In [60]:
# Example of monthly forecast based on correlated features without previous values of the target
data = get_data(Luco_path)
data['Year'] = data.index.year
data = data.resample('M').mean()

target = ['Depth_to_Groundwater_Podere_Casetta']
features = ['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_3', 
            'Volume_Pozzo_1', 'Volume_Pozzo_3', 'Volume_Pozzo_4', 'Year']

# Select input and output columns
data = data[features + target]
# Last row with all features without target
input_date = data.index.max()
input_data = data.loc[input_date, features].values.reshape(1, -1)
# Extract y values
data['Depth_to_Groundwater_Podere_Casetta'] = data['Depth_to_Groundwater_Podere_Casetta'].shift(-1)
data.dropna(inplace=True)
y = data.pop('Depth_to_Groundwater_Podere_Casetta')
# Cross-validate the model
model_name, model = models[0]
cv_r2 = cross_val_score(model, data, y, cv=kf, scoring='r2').mean()
cv_mae = - cross_val_score(model, data, y, cv=kf, scoring='neg_mean_absolute_error').mean()
cv_rmse = - cross_val_score(model, data, y, cv=kf, scoring='neg_mean_squared_error').mean()
print(f'{model_name} cross-val score for {target}:\n\tR2 = {cv_r2}\n\tMAE = {cv_mae}\n\tRMSE = {cv_rmse}')
# Train the model
model.fit(data, y)
prediction = model.predict(input_data)[0]
print(f'\n{model_name} monthly prediction for {target}: {prediction}')
print(f'\nInput features: {", ".join(features)}\nInput date: {input_date}\n')

RandomForest cross-val score for ['Depth_to_Groundwater_Podere_Casetta']:
	R2 = 0.41637576589546815
	MAE = 0.19006197878207873
	RMSE = 0.053600422312016456

RandomForest monthly prediction for ['Depth_to_Groundwater_Podere_Casetta']: -7.533125127334457

Input features: Depth_to_Groundwater_Pozzo_1, Depth_to_Groundwater_Pozzo_3, Volume_Pozzo_1, Volume_Pozzo_3, Volume_Pozzo_4, Year
Input date: 2020-06-30 00:00:00



In [61]:
data = get_data(Petrignano_path)

In [62]:
target_cols = targets['Petrignano']

In [64]:
#data.describe()

In [65]:
#plot_nans(data, 'Petrignano aquifer')

In [66]:
#plot_distribution(data)

In [67]:
#plot_correlation(data, 'Petrignano Aquifer', target_cols)

In [68]:
#plot_timeseries(data, 'Petrignano Aquifer', target_cols)

In [69]:
#plot_seasonality(data, target_cols)

In [70]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [71]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Petrignano Aquifer', 'monthly', 1)


Creating monthly model for Petrignano Aquifer

RandomForest cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9408086795702012
	MAE = 0.5522223755837502
	RMSE = 0.49089810951258855
ExtraTrees cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9435046595570248
	MAE = 0.5136224763971328
	RMSE = 0.4540851411770652
GradientBoosting cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9533088061881175
	MAE = 0.46754929292601627
	RMSE = 0.3985198964490087
KNeighbors cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.8967101614205523
	MAE = 0.733339856500632
	RMSE = 0.9222227055234713

GradientBoosting monthly prediction for Depth_to_Groundwater_P24: -25.079874895867047

Input features: Rainfall_Bastia_Umbra, Depth_to_Groundwater_P24, Depth_to_Groundwater_P25, Volume_C10_Petrignano, Hydrometry_Fiume_Chiascio_Petrignano, Rainfall_Bastia_Umbra_weekly, Year
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.

RandomForest cross-val score for Depth_to_Groundwater_P25:
	R2 

In [72]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Petrignano Aquifer', 'daily', 1)


Creating daily model for Petrignano Aquifer

RandomForest cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9979241059244913
	MAE = 0.10057882636885986
	RMSE = 0.018356249176666676
ExtraTrees cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.997945639512647
	MAE = 0.09994524802871797
	RMSE = 0.018306652428370377
GradientBoosting cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9980265655536512
	MAE = 0.09822997711607702
	RMSE = 0.017576904253079638
KNeighbors cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9943492532107715
	MAE = 0.1581705981934007
	RMSE = 0.05021392395825961

GradientBoosting daily prediction for Depth_to_Groundwater_P24: -25.857758724753662

Input features: Depth_to_Groundwater_P24, Depth_to_Groundwater_P25, Volume_C10_Petrignano, Hydrometry_Fiume_Chiascio_Petrignano, Year
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.

RandomForest cross-val score for Depth_to_Groundwater_P25:
	R2 = 0.9993878257316849
	MAE = 0.05476687898052834
	

In [127]:
data = get_data(Bilancino_path)

In [128]:
target_cols = targets['Bilancino']

In [73]:
#data.describe()

In [74]:
#plot_nans(data, 'Bilancino lake')

In [75]:
#plot_distribution(data)

In [76]:
#plot_correlation(data, 'Bilancino lake', target_cols)

In [77]:
#plot_timeseries(data, 'Bilancino lake', target_cols)

In [78]:
#plot_seasonality(data, target_cols)

In [79]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [80]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Bilancino lake', 'monthly', 1)


Creating monthly model for Bilancino lake

RandomForest cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9391769724750385
	MAE = 0.5640607886487977
	RMSE = 0.5028359347629905
ExtraTrees cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9463864788281361
	MAE = 0.530958297204657
	RMSE = 0.4631036143676777
GradientBoosting cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.953905007139431
	MAE = 0.47602002489596357
	RMSE = 0.3933331817773483
KNeighbors cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.8790584335870117
	MAE = 0.8092610306055062
	RMSE = 1.0709239581702445

GradientBoosting monthly prediction for Depth_to_Groundwater_P24: -25.02137980446495

Input features: Rainfall_Bastia_Umbra, Depth_to_Groundwater_P24, Depth_to_Groundwater_P25, Volume_C10_Petrignano, Hydrometry_Fiume_Chiascio_Petrignano, Rainfall_Bastia_Umbra_weekly, Year, Rainfall_Bastia_Umbra_weekly_weekly
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.

RandomForest cross-val score for 

In [81]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Bilancino lake', 'daily', 1)


Creating daily model for Bilancino lake

RandomForest cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.997938558501502
	MAE = 0.1002102310396436
	RMSE = 0.018117190550829557
ExtraTrees cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9979681008561582
	MAE = 0.09895967045472416
	RMSE = 0.017718707398223028
GradientBoosting cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9979906841025704
	MAE = 0.098535522597415
	RMSE = 0.01761432001583728
KNeighbors cross-val score for Depth_to_Groundwater_P24:
	R2 = 0.9929473867170168
	MAE = 0.17815973981731448
	RMSE = 0.06175661920283717

GradientBoosting daily prediction for Depth_to_Groundwater_P24: -25.84953560470339

Input features: Depth_to_Groundwater_P24, Depth_to_Groundwater_P25, Volume_C10_Petrignano, Hydrometry_Fiume_Chiascio_Petrignano, Year, Rainfall_Bastia_Umbra_weekly_weekly_weekly
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.

RandomForest cross-val score for Depth_to_Groundwater_P25:
	R2 = 0.999393002

In [82]:
data = get_data(Arno_path)

In [83]:
target_cols = targets['Arno']

In [84]:
#data.describe()

In [85]:
#plot_nans(data, 'Arno river')

In [86]:
#plot_distribution(data)

In [87]:
#plot_correlation(data, 'Arno River', target_cols)

In [88]:
#plot_timeseries(data, 'Arno river', target_cols)

In [89]:
#plot_seasonality(data, target_cols)

In [90]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [91]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Arno river', 'monthly', 1)


Creating monthly model for Arno river

RandomForest cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.42985534790430996
	MAE = 0.22380198320002812
	RMSE = 0.08408336442770727
ExtraTrees cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.2986143966834416
	MAE = 0.24692800496352305
	RMSE = 0.10150271031761902
GradientBoosting cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.20387439050810496
	MAE = 0.2547779451607642
	RMSE = 0.11199300678737108
KNeighbors cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.4190545680054589
	MAE = 0.225544012722627
	RMSE = 0.08699976477440152

Average and linear trend monthly prediction for Hydrometry_Nave_di_Rosano: 1.5090329172331394

Input features: Hydrometry_Nave_di_Rosano
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.



In [92]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Arno river', 'daily', 1)


Creating daily model for Arno river

RandomForest cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.804384708275391
	MAE = 0.1214772161263508
	RMSE = 0.060982619931587705
ExtraTrees cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.8253099187154719
	MAE = 0.1180287464671654
	RMSE = 0.057288262160993364
GradientBoosting cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.8083703528285051
	MAE = 0.12358381968493466
	RMSE = 0.062342636316345845
KNeighbors cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.7028165461727003
	MAE = 0.16305251454696593
	RMSE = 0.09488703869492936

ExtraTrees daily prediction for Hydrometry_Nave_di_Rosano: 1.1099999999999997

Input features: Rainfall_Le_Croci, Rainfall_Cavallina, Rainfall_S_Agata, Rainfall_Mangona, Rainfall_S_Piero, Rainfall_Vernio, Rainfall_Stia, Rainfall_Consuma, Rainfall_Incisa, Rainfall_Montevarchi, Rainfall_S_Savino, Rainfall_Laterina, Rainfall_Bibbiena, Rainfall_Camaldoli, Temperature_Firenze, Hydrometry_Nave_di_Rosano,

In [149]:
data = get_data(Amiata_path)

In [150]:
target_cols = targets['Amiata']

In [93]:
#data.describe()

In [94]:
#plot_nans(data, 'Amiata water spring')

In [95]:
#plot_distribution(data)

In [96]:
#plot_correlation(data, 'Amiata water spring', target_cols)

In [97]:
#plot_timeseries(data, 'Amiata water spring', target_cols)

In [98]:
#plot_seasonality(data, target_cols)

In [99]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [100]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Amiata water spring', 'monthly', 1)


Creating monthly model for Amiata water spring

RandomForest cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.442371143720488
	MAE = 0.2219118151851767
	RMSE = 0.08409847341730979
ExtraTrees cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.40506203085583364
	MAE = 0.22984639874362792
	RMSE = 0.08867433888008237
GradientBoosting cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.2813025137786354
	MAE = 0.2500540904348565
	RMSE = 0.10727811757103119
KNeighbors cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.25050668459770975
	MAE = 0.2448135493960283
	RMSE = 0.10521827610033031

Average and linear trend monthly prediction for Hydrometry_Nave_di_Rosano: 1.5090329172331394

Input features: Hydrometry_Nave_di_Rosano
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.



In [101]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Amiata water spring', 'daily', 1)


Creating daily model for Amiata water spring

RandomForest cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.8147356906470908
	MAE = 0.12052216693610368
	RMSE = 0.06141824193833428
ExtraTrees cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.827320020982847
	MAE = 0.11731614678574899
	RMSE = 0.05823015397072851
GradientBoosting cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.8031346737560794
	MAE = 0.12163801445931381
	RMSE = 0.06427623393172628
KNeighbors cross-val score for Hydrometry_Nave_di_Rosano:
	R2 = 0.6989356545142886
	MAE = 0.16212039859009356
	RMSE = 0.0960157149091423

ExtraTrees daily prediction for Hydrometry_Nave_di_Rosano: 1.1099999999999997

Input features: Rainfall_Le_Croci, Rainfall_Cavallina, Rainfall_S_Agata, Rainfall_Mangona, Rainfall_S_Piero, Rainfall_Vernio, Rainfall_Stia, Rainfall_Consuma, Rainfall_Incisa, Rainfall_Montevarchi, Rainfall_S_Savino, Rainfall_Laterina, Rainfall_Bibbiena, Rainfall_Camaldoli, Temperature_Firenze, Hydrometry_Nave_di_

In [102]:
data = get_data(Lupa_path)

In [103]:
target_cols = targets['Lupa']

In [104]:
#data.describe()

In [105]:
#plot_nans(data, 'Lupa water spring')

In [106]:
#plot_distribution(data)

In [107]:
#plot_correlation(data, 'Lupa water spring', target_cols)

In [108]:
#plot_timeseries(data, 'Lupa water spring', target_cols)

In [109]:
#plot_seasonality(data, target_cols)

In [110]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [111]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Lupa water spring', 'monthly', 1)


Creating monthly model for Lupa water spring

RandomForest cross-val score for Flow_Rate_Lupa:
	R2 = 0.8968370059630134
	MAE = 1.7366560153092048
	RMSE = 28.889748586819326
ExtraTrees cross-val score for Flow_Rate_Lupa:
	R2 = 0.9195673423877362
	MAE = 1.2552159220499857
	RMSE = 20.738624310913476
GradientBoosting cross-val score for Flow_Rate_Lupa:
	R2 = 0.9042377504238187
	MAE = 1.640077644652731
	RMSE = 23.130708433090348
KNeighbors cross-val score for Flow_Rate_Lupa:
	R2 = 0.8343482439410335
	MAE = 2.1196851380404067
	RMSE = 52.13929863469363

ExtraTrees monthly prediction for Flow_Rate_Lupa: -78.35032324786323

Input features: Flow_Rate_Lupa, Year
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.



In [112]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Lupa water spring', 'daily', 1)


Creating daily model for Lupa water spring

RandomForest cross-val score for Flow_Rate_Lupa:
	R2 = 0.99981592065491
	MAE = 0.050365653044871526
	RMSE = 0.04100904685139121
ExtraTrees cross-val score for Flow_Rate_Lupa:
	R2 = 0.9998820361217389
	MAE = 0.024687632656699487
	RMSE = 0.026802843227906614
GradientBoosting cross-val score for Flow_Rate_Lupa:
	R2 = 0.9997715317662853
	MAE = 0.11670637061069537
	RMSE = 0.050155255687984124
KNeighbors cross-val score for Flow_Rate_Lupa:
	R2 = 0.9997895802246259
	MAE = 0.047222756410257055
	RMSE = 0.0464949220085469

ExtraTrees daily prediction for Flow_Rate_Lupa: -72.55000000000013

Input features: Flow_Rate_Lupa, Year
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.



In [113]:
data = get_data(Madonna_path)

In [114]:
target_cols = targets['Madonna']

In [115]:
#data.describe()

In [116]:
#plot_nans(data, 'Madonna water spring')

In [117]:
#plot_distribution(data)

In [118]:
#plot_correlation(data, 'Madonna water spring', target_cols)

In [119]:
#plot_timeseries(data, 'Madonna water spring', target_cols)

In [120]:
#plot_seasonality(data, target_cols)

In [121]:
# Feature engineering
data = add_weekly_averages(data)
data = add_seasonality(data)

/Users/gauravtyagi/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [122]:
# Create monthly models and make a forecast
modelling(data, target_cols, 'Madonna di Canneto water spring', 'monthly', 1)


Creating monthly model for Madonna di Canneto water spring

RandomForest cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.15759492224656357
	MAE = 18.239725151197593
	RMSE = 683.3912833700624
ExtraTrees cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.16377185883413226
	MAE = 17.44380616899247
	RMSE = 687.1652309799189
GradientBoosting cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = -0.008368671554179233
	MAE = 19.95670742950143
	RMSE = 842.3749792227673
KNeighbors cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.38692150494015304
	MAE = 16.13776444165276
	RMSE = 513.4777532330578

Average and linear trend monthly prediction for Flow_Rate_Madonna_di_Canneto: 219.2601346618583

Input features: Flow_Rate_Madonna_di_Canneto
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.



In [123]:
# Create daily models and make a forecast
modelling(data, target_cols, 'Madonna di Canneto water spring', 'daily', 1)


Creating daily model for Madonna di Canneto water spring

RandomForest cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.974758943959028
	MAE = 1.773659991191968
	RMSE = 26.236462565779238
ExtraTrees cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.9675039853083365
	MAE = 1.8289005923876591
	RMSE = 31.578047266190396
GradientBoosting cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.9747732579469313
	MAE = 1.653595954300279
	RMSE = 26.01209936393505
KNeighbors cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.9754117781955958
	MAE = 1.6601536684885438
	RMSE = 25.293737935160845

KNeighbors daily prediction for Flow_Rate_Madonna_di_Canneto: 223.70771553999998

Input features: Flow_Rate_Madonna_di_Canneto, Year
Input date: 2020-06-30 00:00:00
Prediction for 1 step(s) ahead.



In [124]:
# Examples of forecasting target values several steps ahead:
for frequency in ['monthly', 'weekly', 'daily']:
    modelling(data, target_cols, 'Madonna di Canneto water spring', frequency, 3)


Creating monthly model for Madonna di Canneto water spring

RandomForest cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = -0.5286909075477607
	MAE = 23.688224668986436
	RMSE = 798.0451741570829
ExtraTrees cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = -0.5789299939366176
	MAE = 22.47669551784287
	RMSE = 792.465501441637
GradientBoosting cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = -0.883692637155125
	MAE = 26.08049883544477
	RMSE = 1048.0304948198527
KNeighbors cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = -0.4181981085116781
	MAE = 21.240171249739152
	RMSE = 692.2501718293279

Average and linear trend monthly prediction for Flow_Rate_Madonna_di_Canneto: 216.70341941906062

Input features: Flow_Rate_Madonna_di_Canneto
Input date: 2020-06-30 00:00:00
Prediction for 3 step(s) ahead.


Creating weekly model for Madonna di Canneto water spring

RandomForest cross-val score for Flow_Rate_Madonna_di_Canneto:
	R2 = 0.5385072454632254
	MAE = 11.74896520631